In [1]:
import cv2
import numpy as np
import os 
import json 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = 'data'

In [3]:
class_list = os.listdir(data)
class_count= len(data)

In [4]:
class_list
#['Apple', 'Banana', 'Lemon', 'Stawberry']

['0', '1', '2', '3']

In [5]:
images = []  # list of images
class_no = [] # list of classes

In [6]:
for x in range (0,class_count):
    myPiclist= os.listdir(data+"/"+str(x))
    for y in myPiclist:
        img = cv2.imread(data+"/"+str(x)+"/"+y)
        img = cv2.resize(img,(32,32))
        images.append(img)
        class_no.append(x)
    
    print(x,"Completed....")

0 Completed....
1 Completed....
2 Completed....
3 Completed....


In [7]:
images = np.array(images)
class_no = np.array(class_no)

# Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [ ]:
help(train_test_split) # more info about spliting

In [10]:
X_train,X_test,y_train,y_test = train_test_split(images,class_no,test_size=0.2)
X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=0.2)

In [11]:
print(X_train.shape) # 3309 train images
print(X_test.shape)  # 1035 test images             # rgb color channel
print(X_validation.shape) # 828 # validation images

(3309, 32, 32, 3)
(1035, 32, 32, 3)
(828, 32, 32, 3)


In [13]:
# Preprossessing function
def prepos(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    
    return img

In [14]:
X_train= np.array(list(map(prePos,X_train)))
X_test= np.array(list(map(prePos,X_test)))
X_validation= np.array(list(map(prePos,X_validation)))

In [15]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
X_validation = X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)

# Preparing the Data for the model

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

In [ ]:
help(ImageDataGenerator)

In [17]:
dataGen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10)
dataGen.fit(X_train)

In [22]:
y_train = to_categorical(y_train,class_count)
y_test = to_categorical(y_test,class_count)
y_validation = to_categorical(y_validation,class_count)

# Creating the Model

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.optimizers import Adam


In [46]:
imageDimensions= (32,32,3)

In [49]:
def myModel():
    noOfFilters = 128
    sizeOfFilter1 = (5,5)
    sizeOfFilter2 = (3, 3)
    sizeOfPool = (2,2)
    noOfNodes= 512
 
    model = Sequential()
    model.add((Conv2D(noOfFilters,sizeOfFilter1,input_shape=(imageDimensions[0],imageDimensions[1],1),activation='relu')))
    model.add((Conv2D(noOfFilters, sizeOfFilter1, activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add((Conv2D(noOfFilters, sizeOfFilter2, activation='relu')))
    model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation='relu')))
    model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add(Dropout(0.2))
 
    model.add(Flatten())
    model.add(Dense(noOfNodes,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(class_count, activation='softmax'))
 
    model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [52]:
model= myModel()
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 28, 28, 128)       3328      
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 24, 24, 128)       409728    
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 10, 10, 128)       147584    
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 8, 8, 64)          73792     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 3, 3, 64)        

In [53]:
## fitting the model
history = model.fit_generator(dataGen.flow(X_train,y_train,
                                 batch_size=16),
                                 steps_per_epoch=100,
                                 epochs=20,
                                 validation_data=(X_validation,y_validation),
                                 shuffle=1)

Epoch 1/20
100/100 [==============================] - 18s 183ms/step - loss: 0.9328 - accuracy: 0.6230 - val_loss: 0.4549 - val_accuracy: 0.7959
Epoch 2/20
100/100 [==============================] - 19s 185ms/step - loss: 0.5022 - accuracy: 0.7969 - val_loss: 0.3710 - val_accuracy: 0.8237
Epoch 3/20
100/100 [==============================] - 19s 185ms/step - loss: 0.4395 - accuracy: 0.8159 - val_loss: 0.2565 - val_accuracy: 0.8563
Epoch 4/20
100/100 [==============================] - 22s 220ms/step - loss: 0.3873 - accuracy: 0.8381 - val_loss: 0.2494 - val_accuracy: 0.9094
Epoch 5/20
100/100 [==============================] - 22s 217ms/step - loss: 0.3293 - accuracy: 0.8619 - val_loss: 0.2705 - val_accuracy: 0.8539
Epoch 6/20
100/100 [==============================] - 22s 216ms/step - loss: 0.2877 - accuracy: 0.8748 - val_loss: 0.2308 - val_accuracy: 0.8829
Epoch 7/20
100/100 [==============================] - 21s 209ms/step - loss: 0.2415 - accuracy: 0.8986 - val_loss: 0.0855 - val_ac

In [54]:
score = model.evaluate(X_test,y_test,verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

Test Score =  0.007817555218935013
Test Accuracy = 0.9971014261245728


In [55]:
# save the model
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('model-bw.h5')